# 介绍
本 `jupyter notebook` 的代码主要是将爬取的 `CSV` 数据转换成 `InteractiveGraph` 所需的 `JSON` 数据。

如果你想构建自己的关系图谱，但不知道如果将手头的数据转换成符合[GitHub - grapheco/InteractiveGraph](https://github.com/grapheco/InteractiveGraph)要求的格式，那么本代码或许能帮助到你，同样的可以看下 `InteractiveGraph` 项目Github里给出的红楼梦数据：[dist/examples/honglou.json](https://github.com/grapheco/InteractiveGraph/blob/master/dist/examples/honglou.json)，非常类似；

如果你只是想运行古柳的明星关系图谱，对前期的数据爬取和转换并不感兴趣，那么跳过这里的代码也是OK的，转换后的 `JSON` 数据就是`webapp\static\ylq_star_relation_graph_v2.json`。

娱乐圈明星关系图谱体验地址：   
https://desertsx.github.io/yulequan-relations-graph/
![](images/show.png)

建议相关文章都读一下，：
- [一文教你用 Neo4j 快速构建明星关系图谱](https://zhuanlan.zhihu.com/p/61096301)  
- [InteractiveGraph 实现酷炫关系图谱之前瞻](https://zhuanlan.zhihu.com/p/63221921)  
- [一文教你使用GitHub Pages部署静态网页](https://zhuanlan.zhihu.com/p/69592043)  
- [安利一个惊艳的红楼梦可视化作品](https://zhuanlan.zhihu.com/p/44584551)   
- [左手读红楼梦，右手写BUG，闲快活](https://zhuanlan.zhihu.com/p/44676544)  

In [68]:
import time
import random
import requests
from lxml import etree
import pandas as pd
from fake_useragent import UserAgent

## 数据来源
`ylq_all_star_relations.csv` 数据集的来源在[一文教你用 Neo4j 快速构建明星关系图谱](https://zhuanlan.zhihu.com/p/61096301)里有介绍，它是爬取自[娱乐圈_专业的娱乐综合门户网站](http://www.ylq.com/)下属“明星”页的“[更多明星](http://www.ylq.com/star/list-all-all-all-all-all-all-all-1.html)”里所有9141条数据后，筛选出个人主页中含“明星关系”的数据，并进一步爬取、解析和存储下的数据集。  

相关代码见于：[DesertsX/gulius-projects/5_YuLeQuan_Neo4j](https://github.com/DesertsX/gulius-projects/tree/master/5_YuLeQuan_Neo4j)。

In [3]:
df_relations = pd.read_csv('ylq_all_star_relations.csv', encoding='utf-8')
df_relations.head()

,num,subject,relation,object,subject_url,object_url,obeject_image,from_id,to_id
0,1,李玥,姐弟,秦俊杰,http://www.ylq.com/neidi/liyue/,http://www.ylq.com/neidi/qinjunjie/,http://img.ylq.com/2016/0527/thumb_200_260_201...,2292,5933
1,2,胡静,同学,刘烨,http://www.ylq.com/neidi/hujing/,http://www.ylq.com/neidi/liuye/,http://img.ylq.com/2014/1203/thumb_200_260_201...,2379,2347
2,2,胡静,同学,秦海璐,http://www.ylq.com/neidi/hujing/,http://www.ylq.com/neidi/qinhailu/,http://img.ylq.com/2017/0224/thumb_200_260_201...,2379,2390
3,2,胡静,同学,袁泉,http://www.ylq.com/neidi/hujing/,http://www.ylq.com/neidi/yuanquan/,http://img.ylq.com/2017/0207/thumb_200_260_201...,2379,2519
4,2,胡静,同学,梅婷,http://www.ylq.com/neidi/hujing/,http://www.ylq.com/neidi/meiting/,http://img.ylq.com/2017/0318/thumb_200_260_201...,2379,2178


In [4]:
df_relations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3433 entries, 0 to 3432
Data columns (total 9 columns):
num              3433 non-null int64
subject          3433 non-null object
relation         3433 non-null object
object           3433 non-null object
subject_url      3433 non-null object
object_url       3433 non-null object
obeject_image    3433 non-null object
from_id          3433 non-null int64
to_id            3433 non-null int64
dtypes: int64(3), object(6)
memory usage: 241.5+ KB


从上面明星关系中筛选去重后，罗列出所有明星的个人主页 URL 列表

In [5]:
star_urls = list(set(df_relations.subject_url.tolist() + df_relations.object_url.tolist()))
len(star_urls)

1282

## 爬取数据
从个人主页中解析出所需的数据，后续关系图谱可视化里想展示哪些数据这里就可以爬取哪些数据。

例如：`infos`数据就是点击图谱中明星结点时会所显示的详细情况，大家完全可以自行完善以展现更详细的数据。

以[刘烨个人主页](http://www.ylq.com/neidi/liuye/)为例，其`infos`就是`['中国吉林', '75 KG', '1978-03-23']`
<img src="images/liuye-homepage.png" width = "900" height = "600">
<img src="images/liuye-graph-node.png" width = "900" height = "600">

In [34]:
%%time
# Wall time: 17min 39s
ylq_star_infos = pd.DataFrame(columns = ['num', 'name', 'relation_num', 'url', 'infos', 'image'])
for num, url in enumerate(star_urls):
    ua = UserAgent()
    headers ={"User-Agent": ua.random, 'Host': 'www.ylq.com'}
    r = requests.get(url=url, headers=headers, timeout=5)
    r.encoding = r.apparent_encoding
    dom = etree.HTML(r.text)
    name = dom.xpath('//div/div/div/h1/text()')[0]
    relations = dom.xpath('//div[@class="hd starRelation"]/ul/li/a/span/em/text()')
    infos = dom.xpath('//div[@class="sLeft"]/ul/li/text()') # infos = ' / '.join(infos)
    image = dom.xpath('///div[@class="sRight"]/a/img/@src')[0]
    data = {'num': int(num+1), 'name': name, 'relation_num': len(relations), 
                'url': url, 'infos': infos, 'image': image}
    ylq_star_infos = ylq_star_infos.append(data, ignore_index=True)
    # print(num, name, url, infos, relations, image)
    print(num, end=' ')  
ylq_star_infos.to_csv('ylq_star_infos.csv', index=False, encoding='utf-8')
print('Finish!')

0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75 76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 91 92 93 94 95 96 97 98 99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177 178 179 180 181 182 183 184 185 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203 204 205 206 207 208 209 210 211 212 213 214 215 216 217 218 219 220 221 222 223 224 225 226 227 228 229 230 231 232 233 234 235 236 237 238 239 240 241 242 243 244 245 246 247 248 249 250 251 252 253 254 255 256 257 258 259 260 261 262 263 264 265 266 267 268 269 270 271 272 273 274 275 276 27

# 处理地区项数据
**本次的关系图谱里涉及两类节点，即：明星类和地区类**，和上述的`infos`数据一样，这里的节点类也可自行扩展至更多。
<img src="images/graph-node.png" width = "900" height = "600">

In [37]:
import pandas as pd
ylq_star_infos = pd.read_csv('ylq_star_infos.csv', encoding='utf-8')
ylq_star_infos.head()

,num,name,relation_num,url,infos,image
0,1,杨志刚,4,http://www.ylq.com/neidi/yangzhigang/,"['中国河北', '68 KG', '1978-11-14']",http://img.ylq.com/2017/0118/thumb_540_400_201...
1,2,胡可,3,http://www.ylq.com/neidi/huke/,"['中国浙江', '45 KG', '1975-12-01']",http://img.ylq.com/2017/0315/thumb_540_400_201...
2,3,黄磊,4,http://www.ylq.com/neidi/huanglei/,"['江西南昌', '不详', '1971-12-06', '不详']",http://img.ylq.com/2014/1203/thumb_540_400_201...
3,4,尹子维,3,http://www.ylq.com/gangtai/yinziwei/,"['中国香港', '70 KG', '1975-05-19']",http://img.ylq.com/2014/1203/thumb_540_400_201...
4,5,刘仁娜,1,http://www.ylq.com/rihan/liurenna/,"['韩国', '44 KG', '1982-06-05', '不详']",http://img.ylq.com/2014/1203/thumb_540_400_201...


列表类的字符串可以用 `eval()` 直接转换成列表

In [53]:
eval("['中国河北', '68 KG', '1978-11-14']")

['中国河北', '68 KG', '1978-11-14']

## 从`infos`列提取出`address`列

In [54]:
ylq_star_infos['address'] = ylq_star_infos.infos.apply(lambda info: eval(info)[0]) # '新西兰,北岛,惠灵顿'
ylq_star_infos.head()

,num,name,relation_num,url,infos,image,address
0,1,杨志刚,4,http://www.ylq.com/neidi/yangzhigang/,"['中国河北', '68 KG', '1978-11-14']",http://img.ylq.com/2017/0118/thumb_540_400_201...,中国河北
1,2,胡可,3,http://www.ylq.com/neidi/huke/,"['中国浙江', '45 KG', '1975-12-01']",http://img.ylq.com/2017/0315/thumb_540_400_201...,中国浙江
2,3,黄磊,4,http://www.ylq.com/neidi/huanglei/,"['江西南昌', '不详', '1971-12-06', '不详']",http://img.ylq.com/2014/1203/thumb_540_400_201...,江西南昌
3,4,尹子维,3,http://www.ylq.com/gangtai/yinziwei/,"['中国香港', '70 KG', '1975-05-19']",http://img.ylq.com/2014/1203/thumb_540_400_201...,中国香港
4,5,刘仁娜,1,http://www.ylq.com/rihan/liurenna/,"['韩国', '44 KG', '1982-06-05', '不详']",http://img.ylq.com/2014/1203/thumb_540_400_201...,韩国


In [59]:
ylq_star_infos['address'].value_counts()

中国              199
韩国               97
中国香港             90
中国台湾             85
日本               57
香港               56
中国北京             38
台湾               32
中国辽宁             29
中国上海             24
美国               20
中国黑龙江            19
中国湖南             19
中国山东             17
中国浙江             15
加拿大              15
中国吉林             14
中国四川             14
中国江苏             14
上海               14
中国重庆             13
北京               12
中国陕西             11
不详               11
中国安徽             10
中国天津              8
中国甘肃              8
马来西亚              8
新加坡               7
中国福建              7
               ... 
香港新界大埔镇泰亨村        1
吉林省通化市            1
文莱                1
中国大连              1
淮安                1
河北唐山              1
四川省广元市            1
河南濮阳市南乐县          1
江西南昌              1
日本香川县善通寺市         1
福州                1
江苏省苏州市姑苏区         1
美国,纽约,锡拉丘兹        1
中国温州              1
澳大利亚新南威尔士冈尼达      1
中国深圳              1
台湾省基隆市            1
上海市徐汇区            1
吉林长               1


In [56]:
address_list = list(set(ylq_star_infos['address'].tolist()))
len(address_list), address_list # 252

(252,
 ['北京市西城区',
  '四川省成都市武侯区',
  '中国',
  '英国,赫特福德郡,斯蒂夫尼奇',
  '中国北京',
  '中国广东省',
  '台湾台北市',
  '中国北京 ',
  '遵义市',
  '中国江苏',
  '中国四川',
  '中国毕竟',
  '山东烟台',
  '浙江金华',
  '山西',
  '美国洛杉矶',
  '美国加利福利亚州洛杉矶',
  '以色列',
  '韩国首尔市',
  '浙江省宁波市',
  '安徽合肥',
  '江苏',
  '中国辽宁',
  '黑龙江',
  '韩国忠清南道天安市新芳洞',
  '英国伯克郡雷丁',
  '浙江省杭州市',
  '杭州',
  '香港新界大埔镇泰亨村',
  '英国英国,英格兰,肯特郡,肖勒姆',
  '新疆乌鲁木齐',
  '中国内蒙古',
  '山西太原',
  '纽约市布朗克斯区',
  '中国 陕西',
  '中国山东',
  '江西省南昌市',
  '上海市徐汇区',
  '中国深圳',
  '陕西延安 ',
  '中国/吉林',
  '澳门',
  '中国青岛',
  '宁夏回族自治区',
  '浙江',
  '澳大利亚,珀斯',
  '广东',
  '南京',
  '重庆渝中区',
  '香港',
  '中国 辽宁',
  '日本宫城县仙台市',
  '成都',
  '中国 天津',
  '齐齐哈尔',
  '香港,模特',
  '中国台湾高雄市左营区',
  '不详',
  '上海市',
  '四川成都',
  '东京',
  '美国,加利福尼亚州,洛杉矶',
  '辽宁省大连市',
  '美国加利福尼亚',
  '中国沈阳',
  '中国 香港',
  '中国 济南',
  '中国重庆',
  '中国山西',
  '广东省广州市白云区',
  '辽宁',
  '台湾省台北市',
  '山东德州',
  '吉林省白山市',
  '韩国首尔',
  '衡水市',
  '美国康涅狄格州费尔菲尔德',
  '美籍华人',
  '安徽安庆',
  '美国加州塔善那',
  '中国 大连',
  '中国福建省龙岩市上杭县',
  '中国江西',
  '台湾省新竹县',
  '江西省宜春市铜鼓县',
  '台湾 ',
  '中国上海',
  '青岛',
 

## 构建更规整的 `area` 列数据
根据自身对后续可视化的想法，可将相同地区的数据统一到一起。

海外的地区一律用对应的国家名；中国的地区有细分的则一律用对应的省份名，无细分的则统一用“中国”；剩下的用“不详”。

下面的`area_list`和`area_map`是根据实际的数据整理出来的，以便后续能用较方便转换和规整数据。

In [57]:
area_list = ['美国', '以色列', '澳大利亚', '英国', '加拿大', '文莱', '新加坡', '西班牙', '越南', '罗马尼亚', '马来西亚', '菲律宾', '新西兰', 
           '韩国', '日本', '北京', '天津', '上海', '重庆', '河北', '山西', '辽宁', '吉林', '江苏', '浙江', '安徽', '福建', '江西', '山东', 
           '河南', '湖北', '湖南', '广东', '海南', '四川', '贵州', '云南', '陕西', '甘肃', '青海', '台湾', '广西', '西藏', '宁夏', '新疆', 
           '香港', '澳门', '内蒙古', '黑龙江']

area_map = {'纽约': '美国', '美籍': '美国', '俄克拉荷马': '美国', '加州': '美国', '伦敦': '英国', 
          '东京': '日本', '京畿道高阳市': '韩国', '大邱广域市': '韩国', '台北':'台湾', '遵义': '贵州',
          '南京': '江苏', '青岛': '山东', '深圳': '广东', '杭州': '浙江', '成都': '四川', '衡水': '河北', 
          '大连': '山东', '齐齐哈尔': '黑龙江', '淮安': '江苏', '温州': '浙江', '唐山': '河北', '福州': '福建', 
          '营口': '辽东', '武汉': '湖北', '广州': '广东'}

def get_city(address):
    for area in area_list:
        if area in address:
            return area
    for area in area_map.keys():
        if area in address:
            return area_map[area]
    if '中国' in address: return '中国'
    else: return '不详'
ylq_star_infos['area'] = ylq_star_infos['address'].apply(get_city)
ylq_star_infos.head()

,num,name,relation_num,url,infos,image,address,area
0,1,杨志刚,4,http://www.ylq.com/neidi/yangzhigang/,"['中国河北', '68 KG', '1978-11-14']",http://img.ylq.com/2017/0118/thumb_540_400_201...,中国河北,河北
1,2,胡可,3,http://www.ylq.com/neidi/huke/,"['中国浙江', '45 KG', '1975-12-01']",http://img.ylq.com/2017/0315/thumb_540_400_201...,中国浙江,浙江
2,3,黄磊,4,http://www.ylq.com/neidi/huanglei/,"['江西南昌', '不详', '1971-12-06', '不详']",http://img.ylq.com/2014/1203/thumb_540_400_201...,江西南昌,江西
3,4,尹子维,3,http://www.ylq.com/gangtai/yinziwei/,"['中国香港', '70 KG', '1975-05-19']",http://img.ylq.com/2014/1203/thumb_540_400_201...,中国香港,香港
4,5,刘仁娜,1,http://www.ylq.com/rihan/liurenna/,"['韩国', '44 KG', '1982-06-05', '不详']",http://img.ylq.com/2014/1203/thumb_540_400_201...,韩国,韩国


In [58]:
ylq_star_infos['area'].value_counts()

中国      211
香港      151
台湾      137
韩国      106
日本       61
北京       61
上海       45
美国       42
辽宁       36
山东       35
湖南       31
浙江       29
黑龙江      27
四川       25
江苏       22
吉林       20
陕西       19
安徽       19
加拿大      17
不详       16
重庆       15
天津       13
湖北       13
河北       11
贵州       11
福建       11
河南       10
广东       10
甘肃       10
马来西亚      8
江西        7
新疆        7
新加坡       7
英国        6
山西        6
云南        4
内蒙古       4
澳门        3
广西        3
越南        2
澳大利亚      2
以色列       1
西班牙       1
菲律宾       1
新西兰       1
西藏        1
罗马尼亚      1
文莱        1
辽东        1
宁夏        1
Name: area, dtype: int64

`area`列共计50个不同的地区

In [67]:
area_counts = ylq_star_infos['area'].value_counts()
area_index = list(area_counts.index)
area_num = list(area_counts.values)
print(len(area_index), len(area_num))
print(area_index)

50 50
['中国', '香港', '台湾', '韩国', '日本', '北京', '上海', '美国', '辽宁', '山东', '湖南', '浙江', '黑龙江', '四川', '江苏', '吉林', '陕西', '安徽', '加拿大', '不详', '重庆', '天津', '湖北', '河北', '贵州', '福建', '河南', '广东', '甘肃', '马来西亚', '江西', '新疆', '新加坡', '英国', '山西', '云南', '内蒙古', '澳门', '广西', '越南', '澳大利亚', '以色列', '西班牙', '菲律宾', '新西兰', '西藏', '罗马尼亚', '文莱', '辽东', '宁夏']


In [ ]:
# ylq_star_infos.to_csv('ylq_star_infos_processed.csv', index=False, encoding='utf-8')

# 构造所需格式的数据
“戏说不是胡说，构造不是瞎造”，关于如何构造出合适的数据格式，可以参考`InteractiveGraph`项目Github里给出的红楼梦数据：[dist/examples/honglou.json](https://github.com/grapheco/InteractiveGraph/blob/master/dist/examples/honglou.json)。我对该数据集曾简单介绍和分析过，可见：[安利一个惊艳的红楼梦可视化作品](https://zhuanlan.zhihu.com/p/44584551)和[左手读红楼梦，右手写BUG，闲快活](https://zhuanlan.zhihu.com/p/44676544)。

另外在本 `Notebook` 末尾也给出了最终的 `JSON` 数据格式，可以先行查看，有所了解。

`dic_categories`里是所有节点类目，如：`{'Star': '明星', 'Area': '地区'}`。包含更多类目的话，都要在此罗列。

## 构造地区节点
遍历所有地区（共50个不同条目）分别构造成字典格式`dict()`，并加入`nodes`列表里，以`北京`为例，构造后的数据格式如下：
```
{'label': '北京',
 'value': 61,
 'id': 117971765168571696383184339562305253550,
 'categories': ['Area'],
 'info': ''},
```

后续明星节点的所有`dict()`同理。涉及多少节点类目，就将所有类目下的数据都通过这种方式加入到`nodes`列表里。

In [160]:
import os
import json
import uuid

dic_categories = {'Star': '明星', 'Area': '地区'}

nodes = []
area2id = {}
for i in range(len(area_index)):
    node = dict()
    node['label'] = area_index[i] # 地区名
    node['value'] = area_num[i] # 出现次数
    node['id'] = int(uuid.uuid1()) # uuid / id
    node['categories'] = ['Area'] # 节点类目
    node['info'] = ''
    area2id[area_index[i]] = node['id'] # 给每个地区名指定 uuid / id
    nodes.append(node)
print(area2id)
nodes

{'中国': 117971764772430883811744432104367026350, '香港': 117971764851659046327010820615000912046, '台湾': 117971764930887208839583212123537109166, '韩国': 117971765010115371354683530397911707822, '日本': 117971765089343533868372251664082366638, '北京': 117971765168571696383184339562305253550, '上海': 117971765247799858897977568637088526510, '美国': 117971765327028021411159916420156715182, '辽宁': 117971765406256183925997900016236982446, '山东': 117971765485484346437381341223147368622, '湖南': 117971765564712508953076979073265004718, '浙江': 117971765643940671469938767751894888622, '黑龙江': 117971765723168833981375689204380299438, '四川': 117971765802396996496812651550900842670, '江苏': 117971765881625159010720360348952391854, '吉林': 117971765960853321526781634193817170094, '陕西': 117971766040081484038437261703206760622, '安徽': 117971766119309646554893163465419878574, '加拿大': 117971766198537809065877754630331265198, '不详': 117971766277765971583666721882246049966, '重庆': 117971766356994134094708170992452989102, '天津': 1179

[{'label': '中国',
  'value': 211,
  'id': 117971764772430883811744432104367026350,
  'categories': ['Area'],
  'info': ''},
 {'label': '香港',
  'value': 151,
  'id': 117971764851659046327010820615000912046,
  'categories': ['Area'],
  'info': ''},
 {'label': '台湾',
  'value': 137,
  'id': 117971764930887208839583212123537109166,
  'categories': ['Area'],
  'info': ''},
 {'label': '韩国',
  'value': 106,
  'id': 117971765010115371354683530397911707822,
  'categories': ['Area'],
  'info': ''},
 {'label': '日本',
  'value': 61,
  'id': 117971765089343533868372251664082366638,
  'categories': ['Area'],
  'info': ''},
 {'label': '北京',
  'value': 61,
  'id': 117971765168571696383184339562305253550,
  'categories': ['Area'],
  'info': ''},
 {'label': '上海',
  'value': 45,
  'id': 117971765247799858897977568637088526510,
  'categories': ['Area'],
  'info': ''},
 {'label': '美国',
  'value': 42,
  'id': 117971765327028021411159916420156715182,
  'categories': ['Area'],
  'info': ''},
 {'label': '辽宁',
  '

## 构造明星节点

In [161]:
ylq_star_infos.head(2)

,num,name,relation_num,url,infos,image,address,area
0,1,杨志刚,4,http://www.ylq.com/neidi/yangzhigang/,"['中国河北', '68 KG', '1978-11-14']",http://img.ylq.com/2017/0118/thumb_540_400_201...,中国河北,河北
1,2,胡可,3,http://www.ylq.com/neidi/huke/,"['中国浙江', '45 KG', '1975-12-01']",http://img.ylq.com/2017/0315/thumb_540_400_201...,中国浙江,浙江


In [162]:
import pandas as pd
df_relations = pd.read_csv('ylq_all_star_relations.csv', encoding='utf-8')
df_relations.head(3)

,num,subject,relation,object,subject_url,object_url,obeject_image,from_id,to_id
0,1,李玥,姐弟,秦俊杰,http://www.ylq.com/neidi/liyue/,http://www.ylq.com/neidi/qinjunjie/,http://img.ylq.com/2016/0527/thumb_200_260_201...,2292,5933
1,2,胡静,同学,刘烨,http://www.ylq.com/neidi/hujing/,http://www.ylq.com/neidi/liuye/,http://img.ylq.com/2014/1203/thumb_200_260_201...,2379,2347
2,2,胡静,同学,秦海璐,http://www.ylq.com/neidi/hujing/,http://www.ylq.com/neidi/qinhailu/,http://img.ylq.com/2017/0224/thumb_200_260_201...,2379,2390


In [163]:
df_relations_small = df_relations[['subject', 'relation', 'object']]
df_relations_small.head(3)

,subject,relation,object
0,李玥,姐弟,秦俊杰
1,胡静,同学,刘烨
2,胡静,同学,秦海璐


In [164]:
ylq_star_infos.shape[0]

1282

https://blog.csdn.net/flyfish5/article/details/79852938  
[Pandas索引和选择数据](https://www.yiibai.com/pandas/python_pandas_indexing_and_selecting_data.html)

In [165]:
# .loc 基于标签(label)，包括行标签(index)和列标签(columns)
# 即行名称和列名称
ylq_star_infos.iloc[2]['name'], ylq_star_infos.loc[2, 'name'] 

('黄磊', '黄磊')

明星类目下的节点构造方式与地区类节点相似，但涉及的数据字段可以不同，比如这里加入图片路径后，图谱里就能展示图片。图片的爬虫由`ylq_star_images_spider.py`完成，此处略过。

输出`nodes`可以看到已经包含了`Area`和`Star`两类节点，后者的格式如下：
```
{'label': '新垣结衣',
  'value': 1,
  'id': 71,
  'image': 'static/images/star/新垣结衣.jpg',
  'categories': ['Star'],
  'info': '日本 / 1988-06-11'},

```

In [166]:
star2id = {}
for idx in range(ylq_star_infos.shape[0]): # 1282
    node = dict()
    star_name = ylq_star_infos.loc[idx, 'name'] # ylq_star_infos.iloc[idx]['name']
    node['label'] = star_name
    node['value'] = ylq_star_infos.loc[idx, 'relation_num']# counter[star_name] # 貌似后者是前者一倍
    node['id'] = idx+1
    node['image'] = "static/images/star/{}.jpg".format(star_name)
    node['categories'] = ['Star']
    infos = eval(ylq_star_infos.loc[idx, 'infos']) # ylq_star_infos.iloc[idx]['infos']
    infos = [info for info in infos if info not in ['0000-00-00', '不详']]
    node['info'] = ' / '.join(infos)
    star2id[star_name] = idx + 1 
    nodes.append(node)
print(len(nodes)) # 1332
nodes

1332


[{'label': '中国',
  'value': 211,
  'id': 117971764772430883811744432104367026350,
  'categories': ['Area'],
  'info': ''},
 {'label': '香港',
  'value': 151,
  'id': 117971764851659046327010820615000912046,
  'categories': ['Area'],
  'info': ''},
 {'label': '台湾',
  'value': 137,
  'id': 117971764930887208839583212123537109166,
  'categories': ['Area'],
  'info': ''},
 {'label': '韩国',
  'value': 106,
  'id': 117971765010115371354683530397911707822,
  'categories': ['Area'],
  'info': ''},
 {'label': '日本',
  'value': 61,
  'id': 117971765089343533868372251664082366638,
  'categories': ['Area'],
  'info': ''},
 {'label': '北京',
  'value': 61,
  'id': 117971765168571696383184339562305253550,
  'categories': ['Area'],
  'info': ''},
 {'label': '上海',
  'value': 45,
  'id': 117971765247799858897977568637088526510,
  'categories': ['Area'],
  'info': ''},
 {'label': '美国',
  'value': 42,
  'id': 117971765327028021411159916420156715182,
  'categories': ['Area'],
  'info': ''},
 {'label': '辽宁',
  '

## 构造明星与明星之间的边关系

In [167]:
df_relations_small.relation.value_counts()

好友      820
绯闻      655
同学      496
拍档      356
旧爱      322
夫妻      203
师徒      132
情侣       98
密友       86
前任夫妻     28
父子       26
组员       26
兄弟       26
搭档       20
同门       16
父女       14
姐妹       14
干父女      12
姐弟       11
干父子      10
母子       10
前任        8
兄妹        6
干母子       6
母女        4
恋人        4
干兄弟       4
准夫妻       2
组合        2
舅甥        2
情人        2
干姐弟       2
翁媳        2
绯闻男友      2
翁婿        2
叔侄        2
父         2
Name: relation, dtype: int64

In [168]:
ylq_star_infos[ylq_star_infos.name == '夏雨'].iloc[0]['num']

54

In [169]:
idx = 14
df_relations_small.loc[idx, 'relation'], df_relations_small.iloc[idx]['relation']

('兄弟', '兄弟')

类似的构造方式，不过不再加入到`nodes`列表里，而是加入到新的`edges`列表里，单条数据的具体格式为：
```
{'id': 221862466013404320763033294366140362926,
  'label': '姐弟',
  'from': 801,
  'to': 1255},
```

In [171]:
edges = []
for idx in range(df_relations_small.shape[0]):
    edge = dict()
    edge['id'] = int(uuid.uuid1())
    edge['label'] = df_relations_small.loc[idx, 'relation'] # df_relations_small.iloc[idx]['relation']
    subject_star = df_relations_small.loc[idx, 'subject']
    object_star = df_relations_small.loc[idx, 'object'] # KeyError:'李蒽熙 ' 可手动去csv里去掉空格变成 '李蒽熙'
    edge['from'] = star2id[subject_star] # ylq_star_infos[ylq_star_infos.name == subject_star].iloc[0]['num']
    edge['to'] = star2id[object_star] # ylq_star_infos[ylq_star_infos.name == object_star].iloc[0]['num']
    # print(edge)
    edges.append(edge)
print(len(edges)) # 3433
edges    

3433


[{'id': 221862466013404320763033294366140362926,
  'label': '姐弟',
  'from': 801,
  'to': 1255},
 {'id': 221863258295029463403509756070547255470,
  'label': '同学',
  'from': 984,
  'to': 514},
 {'id': 221863258374257625920168601290968756398,
  'label': '同学',
  'from': 984,
  'to': 389},
 {'id': 221863258453485788434647422816766226606,
  'label': '同学',
  'from': 984,
  'to': 1049},
 {'id': 221863258532713950948221302292438937774,
  'label': '同学',
  'from': 984,
  'to': 256},
 {'id': 221863258611942113461934793356560134318,
  'label': '同学',
  'from': 984,
  'to': 1020},
 {'id': 221863258691170275975665454394260680878,
  'label': '同学',
  'from': 984,
  'to': 305},
 {'id': 221863258770398438488667376717257339054,
  'label': '同学',
  'from': 984,
  'to': 986},
 {'id': 221863258849626601004719080412913954990,
  'label': '同学',
  'from': 984,
  'to': 67},
 {'id': 221863258928854763519633062252706099374,
  'label': '旧爱',
  'from': 155,
  'to': 454},
 {'id': 221863259008082926031822366317939093678,

## 构造明星与地区之间的边关系
这里统一设置为`出生于`，同样加入到`edges`列表里，具体数据格式为：
```
{
        "id": 300863011087367577823259513777827967150,
        "label": "出生于",
        "from": 42,
        "to": 117971764851659046327010820615000912046
      },
```

In [172]:
for idx in range(ylq_star_infos.shape[0]):
    edge = dict()
    edge['id'] = int(uuid.uuid1())
    edge['label'] = '出生于'
    edge['from'] = idx+1
    area = ylq_star_infos.loc[idx, 'area']
    edge['to'] = area2id[area]
    edges.append(edge)
print(len(edges))
edges

4715


[{'id': 221862466013404320763033294366140362926,
  'label': '姐弟',
  'from': 801,
  'to': 1255},
 {'id': 221863258295029463403509756070547255470,
  'label': '同学',
  'from': 984,
  'to': 514},
 {'id': 221863258374257625920168601290968756398,
  'label': '同学',
  'from': 984,
  'to': 389},
 {'id': 221863258453485788434647422816766226606,
  'label': '同学',
  'from': 984,
  'to': 1049},
 {'id': 221863258532713950948221302292438937774,
  'label': '同学',
  'from': 984,
  'to': 256},
 {'id': 221863258611942113461934793356560134318,
  'label': '同学',
  'from': 984,
  'to': 1020},
 {'id': 221863258691170275975665454394260680878,
  'label': '同学',
  'from': 984,
  'to': 305},
 {'id': 221863258770398438488667376717257339054,
  'label': '同学',
  'from': 984,
  'to': 986},
 {'id': 221863258849626601004719080412913954990,
  'label': '同学',
  'from': 984,
  'to': 67},
 {'id': 221863258928854763519633062252706099374,
  'label': '旧爱',
  'from': 155,
  'to': 454},
 {'id': 221863259008082926031822366317939093678,

# 生成 JSON 文件 
[TypeError: Object of type 'int32' is not JSON serializable ——已解决](https://www.cnblogs.com/lyq-bk1/p/9597172.html)

In [178]:
import numpy as np
class MyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return super(MyEncoder, self).default(obj)

`dic_json = json.dumps(dic, indent=2, ensure_ascii=False)`  
`TypeError: Object of type int64 is not JSON serializable`

在[InteractiveGraph 实现酷炫关系图谱之前瞻](https://zhuanlan.zhihu.com/p/63221921)一文中古柳曾提到原本需要在最终的数据里加一段`JavaScript`代码，但不知道如何写入，为了数据保存时代码缩进等不混乱，所以采取了输出打印数据，然后手动拷贝（`Ctrl+Shift+End`，最后如果有多余的内容，也需要去掉）粘贴的方式，再加上`JavaScript`代码，形成了最终的 `JSON` 数据：`webapp\static\ylq_star_relation_graph_v2.json`。

不过后来[GitHub - grapheco/InteractiveGraph](https://github.com/grapheco/InteractiveGraph)的原作者更新了下代码，应该是不需要加`JavaScript`代码了，可自行尝试，本次略过，那么直接写入 `JSON` 数据即可。

In [179]:
dic = dict()
dic['categories'] = dic_categories
dic['data'] = dict()
dic['data']['nodes'] = nodes
dic['data']['edges'] = edges

dic_json = json.dumps(dic, cls=MyEncoder, indent=2, ensure_ascii=False)
#dic_json = json.dumps(dic, ensure_ascii=False)
print(dic_json)

{
  "categories": {
    "Star": "明星",
    "Area": "地区"
  },
  "data": {
    "nodes": [
      {
        "label": "中国",
        "value": 211,
        "id": 117971764772430883811744432104367026350,
        "categories": [
          "Area"
        ],
        "info": ""
      },
      {
        "label": "香港",
        "value": 151,
        "id": 117971764851659046327010820615000912046,
        "categories": [
          "Area"
        ],
        "info": ""
      },
      {
        "label": "台湾",
        "value": 137,
        "id": 117971764930887208839583212123537109166,
        "categories": [
          "Area"
        ],
        "info": ""
      },
      {
        "label": "韩国",
        "value": 106,
        "id": 117971765010115371354683530397911707822,
        "categories": [
          "Area"
        ],
        "info": ""
      },
      {
        "label": "日本",
        "value": 61,
        "id": 117971765089343533868372251664082366638,
        "categories": [
          "Area"
        ],
  

# JSON 数据最终格式
```
{
  "categories": {
    "Star": "明星",
    "Area": "地区"
  },
  "data": {
    "nodes": [
      {
        "label": "中国",
        "value": 211,
        "id": 117971764772430883811744432104367026350,
        "categories": [
          "Area"
        ],
        "info": ""
      },
      ...
      {
        "label": "新垣结衣",
        "value": 1,
        "id": 71,
        "image": "static/images/star/新垣结衣.jpg",
        "categories": [
          "Star"
        ],
        "info": "日本 / 1988-06-11"
      },
      ...
    ],
    "edges": [
      {
        "id": 221862466013404320763033294366140362926,
        "label": "姐弟",
        "from": 801,
        "to": 1255
      },
      ...
      {
        "id": 300862216428897559752162867914678825134,
        "label": "出生于",
        "from": 1,
        "to": 117971766594678621641213275765944971438
       },
       ...
    ]
  }
}
```